In [ ]:
import os
import json
import numpy as np

def export_edge_metadata(predictor, output_dir="edge_metadata"):
    """Export all necessary metadata for edge deployment"""
    os.makedirs(output_dir, exist_ok=True)
    
    # Custom JSON encoder for NumPy types
    class NumpyEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, (np.integer, np.int64)):
                return int(obj)
            elif isinstance(obj, (np.floating, np.float64)):
                return float(obj)
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            return super(NumpyEncoder, self).default(obj)
    
    # 1. Export service mappings
    with open(os.path.join(output_dir, "service_mappings.json"), "w") as f:
        json.dump(predictor.service_mappings, f, indent=2, cls=NumpyEncoder)
    
    # 2. Export service group stats
    with open(os.path.join(output_dir, "service_group_stats.json"), "w") as f:
        json.dump(predictor.service_group_stats, f, indent=2, cls=NumpyEncoder)
    
    # 3. Export feature list
    feature_info = {
        "basic_features": predictor.basic_features,
        "network_features": predictor.network_features,
        "time_features": predictor.time_features,
        "hour_features": predictor.hour_features,
        "day_features": predictor.day_features,
        "all_features": predictor.features
    }
    with open(os.path.join(output_dir, "feature_list.json"), "w") as f:
        json.dump(feature_info, f, indent=2)
    
    # 4. Export scaler parameters
    scaler_params = {}
    for service_group, scalers in predictor.scalers.items():
        feature_scaler = scalers['features']
        target_scaler = scalers['target']
        
        scaler_params[service_group] = {
            "features": {
                "min": feature_scaler.min_.tolist(),
                "scale": feature_scaler.scale_.tolist()
            },
            "target": {
                "min": target_scaler.min_.tolist(),
                "scale": target_scaler.scale_.tolist()
            }
        }
    with open(os.path.join(output_dir, "scalers.json"), "w") as f:
        json.dump(scaler_params, f, indent=2)
    
    print(f"Exported all metadata to {output_dir}")
    
    # 5. Create README with instructions
    readme = """# Edge Deployment Metadata
    
This directory contains all necessary metadata for deploying bandwidth prediction models on edge devices:

- service_mappings.json: Maps service groups to IDs and services
- service_group_stats.json: Statistical information about service groups
- feature_list.json: List of features used in the models
- scalers.json: Scaling parameters for inputs and outputs

See edge_deployment.py for implementation details.
"""
    with open(os.path.join(output_dir, "README.md"), "w") as f:
        f.write(readme)

To use this function, simply call it after training the models:


In [ ]:
# Train models
predictor = BandwidthAllocationPredictor(dataset_path='network_dataset.csv')
predictor.train_models(epochs=20)

# Export metadata for edge deployment
export_edge_metadata(predictor, "edge_metadata")